#Setup

In [ ]:
!pip install seaborn
!pip install pycaret[full]
!pip  matplotlib==3.1.3
!pip install pycaret-ts-alpha
!pip install scipy>=1.5
!pip install pystan==2.19.1.1
!pip install Cython
!pip install prophet
!pip install -U kaleido
!pip install numpy --upgrade
!cp "/content/drive/MyDrive/Colab Notebooks/Hackaton cajamar 2022/Cajamar_Water_Footprint.zip" "/content/data.zip" #in my case I use drive to upload the file
!unzip -P  UH2022@CwF data

In [ ]:
#@title subir el archivo y descomprimir
do_upload_the_file = False #@param {type:"boolean"}
if do_upload_the_file:
    from google.colab import files

    uploaded = files.upload()

    for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
    !unzip -P  UH2022@CwF data

In [ ]:
%load_ext autoreload

In [ ]:
#@title imports
import pycaret
import pandas as pd
import logging
import matplotlib.pyplot as plt
from tqdm import tqdm
from prophet import Prophet

%matplotlib notebook
%matplotlib inline
import matplotlib
import numpy as np

In [ ]:
target='total'

In [ ]:
def get_range_to_predic(start='2020-01-31',last_date='2020-01-31',periods=1,):
       
        freq='D'
        dates = pd.date_range(
            start=start,
            periods=periods,  # An extra in case we include start
            freq=freq)
    
        return dates
def add_new_date_per_id(df_per_id_but_without_column_id,date,id_unique):
    df_to_add_new_date=pd.DataFrame()
   
    df_to_add_new_date.index=get_range_to_predic(start=date,last_date=date)
    df_to_add_new_date.index = df_to_add_new_date.index.set_names(['date'])
    df_to_add_new_date.reset_index(inplace=True)
    
    df_to_add_new_date=df_to_add_new_date[df_to_add_new_date['date']==date]
      
    return df_to_add_new_date


def get_data():
    list_of_id_with_tiny_data=[2521, 2724, 2725, 2727, 2728, 2729, 2730, 2731, 2732, 2733, 2734, 2735, 2736, 2739, 2743, 2744, 2746, 2747, 2749, 2756]
    data_path='/content/Modelar_UH2022.txt'
    df=pd.read_csv( data_path,sep="|")
    df=df[df['ID']<=25]
    # df=df[df['ID'].isin(list_of_id_with_tiny_data)]
    logging.info(df.shape)
    df.drop(columns=['READINGINTEGER','READINGTHOUSANDTH'],inplace=True)
    df.fillna(0,inplace=True) #check in future
    logging.info(df.shape)
    df.loc[:,'SAMPLETIME']=pd.to_datetime(df['SAMPLETIME'],errors='raise',format='%Y-%m-%d')
    # df['target']
    print(df.columns)
    df.rename(columns={'SAMPLETIME':'date'},inplace=True)
    df['total']=df['DELTAINTEGER']+df['DELTATHOUSANDTH']
    df.drop(['DELTAINTEGER','DELTATHOUSANDTH'],axis=1,inplace=True)
    print(df.columns)
    # df=df.groupby(['ID','year','month','day'])
    # df=df[target].sum()
    # df=df.reset_index()
    # df['date']=pd.to_datetime(df[['year', 'month', 'day']])

    df_total=pd.DataFrame()
    for id_unique in tqdm(df.ID.unique(),desc='creating all the dates per ID',mininterval=50):
        df_per_id=df[df['ID']==id_unique]
        df_per_id_but_without_column_id=df_per_id.drop('ID',axis=1)
        list_of_df_with_new_dates=[]
        date='2020-01-31'
        if not df_per_id.date.isin([date]).any():
            # print('adding',date)
            df_with_last_date=add_new_date_per_id(df_per_id_but_without_column_id,date,id_unique)
            list_of_df_with_new_dates.append(df_with_last_date)
        else:print(id_unique,'ya tiene la fecha',date)
        date='2019-02-01'
        if not df_per_id.date.isin([date]).any():
            # print('adding',date)
            df_with_start_date=add_new_date_per_id(df_per_id_but_without_column_id,date,id_unique)
            list_of_df_with_new_dates.append(df_with_start_date)
        else:print(id_unique,'ya tiene la fecha',date)
        if list_of_df_with_new_dates:
            # print('list_of_df_with_new_dates')
            df_aux=pd.concat([df_per_id_but_without_column_id,*list_of_df_with_new_dates])
        else:    
            df_aux=df_per_id_but_without_column_id
            print('no_new_date')
        # df_total.drop('ID',inplace=True,axis=1)
        df_aux=df_aux.resample('D',on='date').sum()
        df_aux['ID']=id_unique
        df_total=pd.concat([df_total,df_aux])
    return df_total.reset_index()

def plot_series(time, series,i, format="-", start=0, end=None):

    
    
    plt.plot(time[start:end], series[start:end], format,label=i)
    plt.xlabel("Date")
    plt.ylabel("Sales (Water)")
    plt.legend()

#EDA

##descarga de datos

In [ ]:
data_path='/content/Modelar_UH2022.txt'
df_raw=pd.read_csv( data_path,sep="|")
print(df_raw.shape)
df_raw.dtypes


In [ ]:
df_raw.loc[:,'SAMPLETIME']=pd.to_datetime(df_raw['SAMPLETIME'],errors='raise',format='%Y-%m-%d %H:%M:%S')

In [ ]:
df_raw.info()

In [ ]:
df_raw.head()

In [ ]:
df_raw.describe()

##Check duplicated

In [ ]:
df_raw.duplicated(['ID','SAMPLETIME']).sum()


No existen duplicados

Con esto se ha descubierto que tenemos números negativos, los cuales no son posibles si es unontador de agua, por lo que analizaremos cuantos y el motivo para remediarlo

##casos negativos

In [ ]:
negative_cases=df_raw[df_raw['DELTAINTEGER']<0].sort_values(['ID','SAMPLETIME'])

In [ ]:
unique_negative_id=negative_cases.ID.unique()
len(unique_negative_id)

Con esto se observa que tenemos poco más de 8500 registros negativos comprendidos en unos casi 500 ID diferentes, vamos a analizar que ocurre para solventarlo

In [ ]:
df_raw[(df_raw['ID']==3)&(df_raw['SAMPLETIME'].dt.month==5)&(df_raw['SAMPLETIME'].dt.day.isin([11,10]))].sort_values(['ID','SAMPLETIME'])

Se observa en este caso aunque exista el -1 (probablemente por alguna regulación del contador) no afecta, vamos a analizar casos más extremos

In [ ]:
print(df_raw[(df_raw['ID']==2711)].sort_values(['ID','SAMPLETIME']).tail())

In [ ]:
df_raw[(df_raw['ID']==2711)&(df_raw['SAMPLETIME'].dt.month==12)&(df_raw['SAMPLETIME'].dt.day.isin([15,16]))].sort_values(['ID','SAMPLETIME'])

In [ ]:
df_raw[(df_raw['ID']==57)&(df_raw['SAMPLETIME'].dt.month==5)&(df_raw['SAMPLETIME'].dt.day.isin([18,17]))].sort_values(['ID','SAMPLETIME'])

Suponemos que son regularizaciones aplicadas en el contador, igualmente se les ha mandado una pregunta al equipo

Pero antes de eso vamos a ver cuales son lo smayores valores negativos

In [ ]:
df_raw[df_raw['DELTAINTEGER']<0].sort_values(['ID','SAMPLETIME']).groupby('ID')['DELTAINTEGER'].sum().sort_values(ascending=True).head(15)

In [ ]:
df_raw[df_raw['DELTAINTEGER']<0].sort_values(['ID','SAMPLETIME']).groupby('ID')['DELTAINTEGER'].count().sort_values(ascending=False).head(15)

In [ ]:
df_raw[(df_raw['ID']==2063)]['DELTAINTEGER'].plot()

In [ ]:
df_raw[(df_raw['ID']==2063)&(df_raw['DELTAINTEGER']<0)].sort_values(['ID','SAMPLETIME'])

In [ ]:
df_raw[(df_raw['ID']==2063)&(df_raw['SAMPLETIME'].dt.month==9)&(df_raw['SAMPLETIME'].dt.day.isin([14,15]))].sort_values(['ID','SAMPLETIME'])

al agruparlo por día desaparecera el valor erroneo aunque puede que no en todos los casos
Igualmente vamos a gráficarlos y analizarlo más en detale


In [ ]:
#@title plot negative id per year
id_per_plot = 1 #@param {type:"number"}
plt.figure(figsize=(20,5))
for i, id_unique in tqdm(enumerate(unique_negative_id),desc="creating plots"):
    
    df_aux=df_raw[df_raw['ID'] == id_unique].sort_values(['ID','SAMPLETIME'])
    # bin=df_aux['group_by_mean_in_total'].unique()[0]
    plot_series(df_aux['SAMPLETIME'],df_aux['DELTAINTEGER'],id_unique)
    min=df_aux['DELTAINTEGER'].min()
    if i %id_per_plot==0:
        plt.ylim(min-5,abs(min)+5)
        plt.title(f'This id is {str(id_unique)}')
        plt.axhline(y = 0, color = 'k', linestyle = '-')
        plt.show()
        plt.figure(figsize=(20,5))

# 57, 301,374,379,493,635,
#845 (un pico negativo que puede ocupar todo el vació que ha dejado por algún fallo de algo)
#873, 907,1218,1280,1468,1487,1506,1739,1873,1884, 2063,2121
#2711

Dentro de los casi 500 id con valores negativos hay que darles especial atención a los siguientes 57, 301,374,379,493,635,845 873, 907,1218,1280,1468,1487,1506,1739,1873,1884, 2063,2121, 2711
Vamos a analizarlos en profundidad y ver que tipo de problemas podemos encontrarnos


In [ ]:
#@title plot special negative id per year
id_per_plot = 1 #@param {type:"number"}
plt.figure(figsize=(20,5))
special_unique_negative_id=[57, 301,374,379,493,635,845,
                            873, 907,1218,1280,1468,1487,
                            1506,1739,1873,1884, 2063,2121,
                            2711
                            ]
for i, id_unique in tqdm(enumerate(special_unique_negative_id),desc="creating plots"):
    
    df_aux=df_raw[df_raw['ID'] == id_unique].sort_values(['ID','SAMPLETIME'])
    # bin=df_aux['group_by_mean_in_total'].unique()[0]
    plot_series(df_aux['SAMPLETIME'],df_aux['DELTAINTEGER'],id_unique)
    min=df_aux['DELTAINTEGER'].min()
    if i %id_per_plot==0:
        plt.ylim(min-5,abs(min)+5)
        plt.title(f'This id is {str(id_unique)}')
        plt.axhline(y = 0, color = 'k', linestyle = '-')
        plt.show()
        plt.figure(figsize=(20,5))

Algunos d esos problemas son: 
* El 2711 está a la inversa por lo que hay que poner todo en valor absoluto, ayudanod de esta forma a la estacionalidad.
* Picos con valores muy altos/bajos que se compensan entre ellos como si fuera un error de lectura y luego vuelve al valor correspondiente 57,635,873,907,1218,1280,2063 estos pondremos los picos directamente a 0, ya que además casi todos los puntos son 0
* El contador ha dejado de grabar durante un periodo de tiempo y aparece como negativo el primer valor, pero probablemente es el conjunto de valores anteriores, es el caso del 845 en este caso en concreto, después de ver la gráfica le daremos el valor de la media a los registros de valor 0

* Y por último algún que otro posible pico negativo aleatorio, sin encontrar explicación ninguna, estos otros le daremos el valor de la media


##Exceso de datos
Exceso de datos, por lo que eliminamos las columnasreading y solo usamos las delta, además transformamos ambas columnas delta a una sola, donde basicamente se añaden los decimales. 


In [ ]:
df_raw.drop(columns=['READINGINTEGER','READINGTHOUSANDTH'],inplace=True)
df_raw.fillna(0,inplace=True) #check in future
logging.info(df_raw.shape)
df_raw.loc[:,'SAMPLETIME']=pd.to_datetime(df_raw['SAMPLETIME'],errors='raise',format='%Y-%m-%d')
# df['target']
print(df_raw.columns)
df_raw.rename(columns={'SAMPLETIME':'date'},inplace=True)
df_raw['total']=df_raw['DELTAINTEGER']+df_raw['DELTATHOUSANDTH']
df_raw.drop(['DELTAINTEGER','DELTATHOUSANDTH'],axis=1,inplace=True)
df_raw.head()

In [ ]:
df_raw.describe()

Destacar que se siguen obteniendo valores negativos,

No corre en colab

In [ ]:
#@title Pivot table
is_colab = True #@param {type:"boolean"}
if not is_colab:
    pivot_by_dates=df_raw.pivot(
        index='date',
        columns='ID',
        values='total'
    )
    pivot_by_ID=df_raw.pivot(
        index='ID',
        columns='date',
        values='total'
    )

##Todos tienen las mismas fechas, existen dias que no nos dan datos?

Lo ideal es agrupar por dias o por semanas ya que es el objetivo de la competición siendo la fecha inicial el 1 de febrero de 2019 y la fecha final el 31 de enero de 2020. Para ello añadimos la primera y la última fecha a cada uno de los ID para luego realizar un resample por cada día

In [ ]:
df_total=pd.DataFrame()
for id_unique in tqdm(df_raw.ID.unique(),desc='creating all the dates per ID',mininterval=50):
    df_per_id=df_raw[df_raw['ID']==id_unique]
    df_per_id_but_without_column_id=df_per_id.drop('ID',axis=1)
    list_of_df_with_new_dates=[]
    date='2020-01-31'
    if not df_per_id.date.isin([date]).any():
        # print('adding',date)
        df_with_last_date=add_new_date_per_id(df_per_id_but_without_column_id,date,id_unique)
        list_of_df_with_new_dates.append(df_with_last_date)
    # else:print(id_unique,'ya tiene la fecha',date)
    date='2019-02-01'
    if not df_per_id.date.isin([date]).any():
        # print('adding',date)
        df_with_start_date=add_new_date_per_id(df_per_id_but_without_column_id,date,id_unique)
        list_of_df_with_new_dates.append(df_with_start_date)
    # else:print(id_unique,'ya tiene la fecha',date)
    if list_of_df_with_new_dates:
        # print('list_of_df_with_new_dates')
        df_aux=pd.concat([df_per_id_but_without_column_id,*list_of_df_with_new_dates])
    else:    
        df_aux=df_per_id_but_without_column_id
        # print('no_new_date')
    # df_total.drop('ID',inplace=True,axis=1)
    df_aux=df_aux.resample('D',on='date').sum()
    df_aux['ID']=id_unique
    df_total=pd.concat([df_total,df_aux])

In [ ]:
print(df_total.head())
df_total.shape

In [ ]:
df_total.describe()

###exceso de negativos, casos especiales

Analizamos como se portan los casos negativos especiales

In [ ]:
#@title plot special negative id per year
id_per_plot = 1 #@param {type:"number"}
plt.figure(figsize=(20,5))
special_unique_negative_id=[57, 301,374,379,493,635,845,
                            873, 907,1218,1280,1468,1487,
                            1506,1739,1873,1884, 2063,2121,
                            2711
                            ]
for i, id_unique in tqdm(enumerate(special_unique_negative_id),desc="creating plots"):
    
    df_aux=df_total[df_total['ID'] == id_unique].sort_values(['ID','date'])
    # bin=df_aux['group_by_mean_in_total'].unique()[0]
    plot_series(df_aux.index,df_aux['total'],id_unique)
    min=df_aux['total'].min()
    if i %id_per_plot==0:
        plt.ylim(min-5,abs(min)+5)
        plt.title(f'This id is {str(id_unique)}')
        plt.axhline(y = 0, color = 'k', linestyle = '-')
        plt.show()
        plt.figure(figsize=(20,5))

#existe outliers positivos??
Se utilizo un insolation forest para tratar esto

In [ ]:
id_researching=range(0,20)
from sklearn.ensemble import IsolationForest
#no modificar los días después del 22 de enero


outliers_fraction=0.01
method_forest=IsolationForest(contamination=outliers_fraction)
df_after_outliers=pd.DataFrame()
plt.figure(figsize=(20,5))
for i,id_unique in enumerate(id_researching):
    df_per_id=df_total[df_total['ID']==id_unique].copy()
    
    df_per_id.head(20)
    yhat = method_forest.fit_predict(df_per_id['total'].to_numpy().reshape(-1,1))
    mask = yhat != -1
    plot_series(df_per_id.index,df_per_id['total'],f'todo {id_unique}')
    plot_series(df_per_id[mask].index,df_per_id[mask]['total'],f'true {id_unique}')
    plot_series(df_per_id[~mask].index,df_per_id[~mask]['total'],f'mask {id_unique}')
    plt.title('original')
    plt.show()
    plt.figure(figsize=(20,5))
    print('\n')
    df_per_id['total']=np.where((mask) | (df_per_id.index> '2020-01-22'),df_per_id['total'],
            df_per_id['total'].shift(periods=7).fillna(method='bfill')
                    
                    )
    plot_series(df_per_id.index,df_per_id['total'],f'todo {id_unique}')
    plot_series(df_per_id[mask].index,df_per_id[mask]['total'],f'true {id_unique}')
    plot_series(df_per_id[~mask].index,df_per_id[~mask]['total'],f'mask {id_unique}')
    plt.title('modified')
    plt.show()
    plt.figure(figsize=(20,5))
    df_after_outliers=pd.concat([df_after_outliers,df_per_id])
    print('\n'*3)
    if i>20:
        break

###Working with pivots table

In [ ]:
df_total_to_pivot=df_total.reset_index()
df_total_to_pivot

In [ ]:
pivot_by_dates=df_total_to_pivot.pivot(
    index='date',
    columns='ID',
    values='total'
)
pivot_by_dates

In [ ]:
import seaborn as sns

#drawing one figure with title and 3 axes below each other. Size and resolution are specified
f, axes = plt.subplots(4,1,figsize=(18,12),dpi=600,sharex=True)
plt.suptitle('Comparison of unscaled features at different scales',fontsize=22);

#drawing boxplots of three different scales, each to separate axis
max_id=100
sns.boxplot(data=pivot_by_dates.iloc[:,1:max_id], ax=axes[0])
sns.boxplot(data=pivot_by_dates.iloc[:,1:max_id], ax=axes[1]).set(ylim=(0,5000))
sns.boxplot(data=pivot_by_dates.iloc[:,1:max_id], ax=axes[2]).set(ylim=(0,2500))
sns.boxplot(data=pivot_by_dates.iloc[:,1:max_id], ax=axes[3]).set(ylim=(0,1000))

#rotating ticks of the shared x axis by 90 degrees. The shared x axis is located on axes[2]
for tick in axes[3].get_xticklabels():
        tick.set_rotation(90);

#setting y axis labels for each axis
for a in axes:
    a.set_ylabel('Unscaled values');

Con estos primeros 100 ID se puede observar que el rango de cada ID es muy variado, , desde 10000 de media algunos hasta muchos entre valores comprendidos menores a 1000. Se recomienda analizar el resto de ID de una forma más descriptiva, sin gráfica ya que esto solo representa el 5% aproximadamente de los datos

In [ ]:
describe_by_dates=pivot_by_dates.describe()
describe_by_dates

In [ ]:
describe_by_dates.T

In [ ]:
import seaborn as sns

#drawing one figure with title and 3 axes below each other. Size and resolution are specified
f, axes = plt.subplots(4,1,figsize=(18,12),dpi=600,sharex=True)
plt.suptitle('Comparison of unscaled features at different scales',fontsize=22);

#drawing boxplots of three different scales, each to separate axis
max_id=100
sns.boxplot(data=describe_by_dates.T, ax=axes[0]).set(ylim=(-5000,20000))
sns.boxplot(data=describe_by_dates.T, ax=axes[1]).set(ylim=(0,5000))
sns.boxplot(data=describe_by_dates.T, ax=axes[2]).set(ylim=(0,2500))
sns.boxplot(data=describe_by_dates.T, ax=axes[3]).set(ylim=(0,1000))

#rotating ticks of the shared x axis by 90 degrees. The shared x axis is located on axes[2]
for tick in axes[3].get_xticklabels():
        tick.set_rotation(90);

#setting y axis labels for each axis
for a in axes:
    a.set_ylabel('Unscaled values');

De esta forma se observa que la mayoría de los id tienen valores similares, con un rango intercuartilico de una amplitud de 500 en la media y desviación típica. Igual que el valor máximo tiene una gran amplitud de valores posibles y que el mínimo de normal es 0

In [ ]:
pivot_by_id=df_total_to_pivot.pivot(
    index='ID',
    columns='date',
    values='total'
)
pivot_by_id

In [ ]:
import seaborn as sns

#drawing one figure with title and 3 axes below each other. Size and resolution are specified
f, axes = plt.subplots(4,1,figsize=(18,12),dpi=600,sharex=True)
plt.suptitle('Comparison of unscaled features at different scales',fontsize=22);

#drawing boxplots of three different scales, each to separate axis
max_id=90 #3 meses
sns.boxplot(data=pivot_by_id.iloc[:,1:max_id], ax=axes[0])
sns.boxplot(data=pivot_by_id.iloc[:,1:max_id], ax=axes[1]).set(ylim=(0,5000))
sns.boxplot(data=pivot_by_id.iloc[:,1:max_id], ax=axes[2]).set(ylim=(0,2500))
sns.boxplot(data=pivot_by_id.iloc[:,1:max_id], ax=axes[3]).set(ylim=(0,1000))

#rotating ticks of the shared x axis by 90 degrees. The shared x axis is located on axes[2]
for tick in axes[3].get_xticklabels():
        tick.set_rotation(90);

#setting y axis labels for each axis
for a in axes:
    a.set_ylabel('Unscaled values');

Con esta gáfica observamos que son los ID son principalmente quienes nos dan información, aunque también se puede sacar dos principales conclusiones. 


*   La primera de ellas es que la mayoría de ID tienen valores comprendidos entre 0 y 400
*   La segunda es el efecto estacional, por ejemplo el 6 ed marzo se observa un incremento del consumo,que más o menos coincide con el inicio de las fallas de ese año. como en nuestro estudio coincide con el covid y que es para febrero eso no lo tendremos en cuenta.

En relación a la estacionalidad la analizaremos más adelante



In [ ]:
describe_by_id=pivot_by_id.describe()
describe_by_id

In [ ]:
import seaborn as sns

#drawing one figure with title and 3 axes below each other. Size and resolution are specified
f, axes = plt.subplots(4,1,figsize=(18,12),dpi=600,sharex=True)
plt.suptitle('Comparison of unscaled features at different scales',fontsize=22);

#drawing boxplots of three different scales, each to separate axis
max_id=100
sns.boxplot(data=describe_by_id.T, ax=axes[0]).set(ylim=(-5000,20000))
sns.boxplot(data=describe_by_id.T, ax=axes[1]).set(ylim=(-500,5000))
sns.boxplot(data=describe_by_id.T, ax=axes[2]).set(ylim=(-250,2500))
sns.boxplot(data=describe_by_id.T, ax=axes[3]).set(ylim=(-100,1000))

#rotating ticks of the shared x axis by 90 degrees. The shared x axis is located on axes[2]
for tick in axes[3].get_xticklabels():
        tick.set_rotation(90);

#setting y axis labels for each axis
for a in axes:
    a.set_ylabel('Unscaled values');

Con esta gráfica podemos observar la diferencia entre los valores de las distintas fechas,variando entre 400 y 800 y donde tenemos algunos valores atípicos en la media (observable en la siguiente gráfica), probablemente por algún ID que tendrá algún valor anómalo. Además destacar la gran diferencia entre algunos ID provoca que la desviación stándar sea enorme

In [ ]:
plot_series(describe_by_id.T.index,describe_by_id.T['mean'],'mean')

##Creating groups of ID per mean

Empezamos creando dos grupos de ID iniciales, el primero de ellos mayor a 2000 y el segundo entre 1000 a 2000

In [ ]:
group_2000=describe_by_dates.T[describe_by_dates.T['mean']>2000]
group_1000_2000=describe_by_dates.T[(describe_by_dates.T['mean']>1000)&(describe_by_dates.T['mean']<2000)]
print('>2000',group_2000.shape)
print('1000>ID<2000',group_1000_2000.shape)
rest_df=describe_by_dates.T[describe_by_dates.T['mean']<=1000]
rest_df

In [ ]:
rest_df.describe()

Ya que los ID tienen tanta variabilidad entre ellos se ha decidido agrupar por grupos

In [ ]:
sns.distplot(
    rest_df['mean'], norm_hist=False, kde=False, bins=50, hist_kws={"alpha": 1}
).set(xlabel='Sale Price', ylabel='Count');

In [ ]:
rest_df['group_by_mean_in_total']=pd.qcut(rest_df['mean'],q=50,labels=range(50))
group_1000_2000['group_by_mean_in_total']=50
group_2000['group_by_mean_in_total']=51

In [ ]:
df_with_group_by_mean_in_total=pd.concat([rest_df,group_1000_2000,group_2000])['group_by_mean_in_total']
df_with_group_by_mean_in_total.head()

In [ ]:
df_with_group_by_mean_in_total.to_csv('group_by_mean_in_total.csv')

##Plotting a fews plots

In [ ]:
df_total_with_bins=pd.merge(df_with_group_by_mean_in_total,df_total,how='right',left_index=True,right_on='ID')


In [ ]:
#@title plot id per year
id_per_plot = 10 #@param {type:"number"}
plt.figure(figsize=(20,5))
for i, id_unique in tqdm(enumerate(df_total_with_bins.sort_values(['group_by_mean_in_total','date']).ID.unique()),desc="creating plots"):
    
    df_aux=df_total_with_bins[df_total_with_bins['ID'] == id_unique]
    bin=df_aux['group_by_mean_in_total'].unique()[0]
    plot_series(df_aux.index,df_aux['total'],id_unique)
    
    if i %id_per_plot==0:
        plt.title(f'This bin {str(bin)}')
        plt.show()
        plt.figure(figsize=(20,5))
    

En las gráficas se ve que el id 353 y 2711 hay que corregirlos, siendo en el 353 poniendo los picos a 0 y en el 2711 probablemente poniendolo en positivo el valor negativo y recalcular lo de los bins

##Análisis de dependencia por correlación entre los distintos ID

In [ ]:
pivot_by_dates.head()

In [ ]:

corr_matrix = pivot_by_dates.corr()

corr = ((corr_matrix + corr_matrix.T)/2 ).values                        # made symmetric
np.fill_diagonal(corr, 1)  

In [ ]:
import numpy as np
data_to_correlation=pivot_by_dates.copy()
# data_to_correlation.drop(['ID'],axis=1,inplace=True)
corr_matrix = data_to_correlation.corr()
 # Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
s = upper.unstack()
so = s.sort_values(kind="quicksort")
to_use_in_next_study = so[so>0.8]
to_use_in_next_study

In [ ]:
plt.figure(figsize=(20,5))
# ids=[2733,2727,2735,2731,2736  ]
# ids=[2739,2735,2733,2731,2736]
# ids=[1498,42]
# ids=[87,21]
ids=[2652,2658]
for ids,corr in to_use_in_next_study.iteritems():
    print(ids,corr)
    for i, id_unique in tqdm(enumerate(df_total_with_bins[df_total_with_bins.ID.isin(ids)].sort_values(['group_by_mean_in_total','date']).ID.unique()),desc="creating plots"):
        print(id_unique)
        df_aux=df_total_with_bins[df_total_with_bins['ID'] == id_unique]
        bin=df_aux['group_by_mean_in_total'].unique()[0]
        plot_series(df_aux.index,df_aux['total'],id_unique)
        
        # if i %id_per_plot==0 and i>0:
    plt.title(f'This bin {str(bin)}')
    plt.show()
    plt.figure(figsize=(20,5))

Esto nos da información sobre variables que son similares a otras variables, por lo que deberíamos de utilizar esta información en el modelo en algún momento

In [ ]:
so.head(50)

##trends and stacionality

###using statsmodels

In [ ]:
import statsmodels.api as sm
from matplotlib import rcParams


In [ ]:
pivot_by_dates[2].plot()

In [ ]:
id_per_plots=1
plt.figure(figsize=(20,15))
for i,ts in enumerate(pivot_by_dates.columns):
    decomposition = sm.tsa.seasonal_decompose(pivot_by_dates[ts].dropna())
    # Store back the results
    data = decomposition.seasonal
    plot_series(data.index,data.values,ts)
    if i%id_per_plots==0 and i>=id_per_plots:

        plt.title(f'id between {str(i)}')
        plt.show()
        plt.figure(figsize=(20,15))
    if i >=100:
        break

In [ ]:
id_per_plots=10
plt.figure(figsize=(20,15))
for i,ts in enumerate(pivot_by_dates.columns):
    decomposition = sm.tsa.seasonal_decompose(pivot_by_dates[ts].dropna())
    # Store back the results
    data = decomposition.trend
    plot_series(data.index,data.values,ts)
    if i%id_per_plots==0 and i>8:

        plt.title(f'id between {str(i)}')
        plt.show()
        plt.figure(figsize=(20,15))
    if i >=100:
        break

In [ ]:
id_per_plots=10
plt.figure(figsize=(20,15))
for i,ts in enumerate(pivot_by_dates.columns):
    decomposition = sm.tsa.seasonal_decompose(pivot_by_dates[ts].dropna())
    # Store back the results
    data = decomposition.resid
    plot_series(data.index,data.values,ts)
    if i%id_per_plots==0 and i>8:

        plt.title(f'id between {str(i)}')
        plt.show()
        plt.figure(figsize=(20,15))
    if i >=100:
        break

In [ ]:
from statsmodels.graphics import tsaplots
id_per_plots=10
plt.figure(figsize=(20,15))
for i,ts in enumerate(pivot_by_dates.columns):
    tsaplots.plot_acf(pivot_by_dates[ts], lags=90)
    
    plt.xlabel("Lag at k")
    plt.ylabel("Correlation coefficient")
    plt.title(f'id between {str(i)}')
    plt.show()
    plt.figure(figsize=(20,5))
    if i >=100:
        break

In [ ]:
from statsmodels.tsa.seasonal import STL
id_per_plots=10
for i,ts in enumerate(pivot_by_dates.columns):
    res=STL(pivot_by_dates[ts],).fit()
    fig=res.plot()
    fig.set_figheight(10)
    fig.set_figwidth(20)
    plt.xlabel("Lag at k")
    plt.ylabel("Correlation coefficient")
    plt.title(f'id between {str(i)}')
    plt.show()
    print('\n')
    if i >=100:
        break

In [ ]:
a

###using prophet

In [ ]:
pivot_by_dates.plot()

In [ ]:
df_prophet=pd.DataFrame()
df_aux=pivot_by_dates[81]
df_prophet[['ds','y']]=df_aux.reset_index()

In [ ]:
df_prophet

Solo mostramos los 20 primeros

In [ ]:
for i,ts in tqdm(enumerate(pivot_by_dates.columns)): 
    df_prophet=pd.DataFrame()
    df_aux=pivot_by_dates[ts]
    df_prophet[['ds','y']]=df_aux.reset_index()
    m = Prophet(yearly_seasonality=False,
        weekly_seasonality=True,
        daily_seasonality=False
        )
    m.fit(df_prophet)
    forecast = m.predict(df_prophet)
    fig1 = m.plot(forecast)
    fig1.suptitle('id '+str(ts))
    fig1.show()
    print("\n"*1)
    fig2 = m.plot_components(forecast)
    fig2.suptitle('id '+str(ts))
    print("\n"*3)
    if i>20:
        break

### with pycaret --> time series

In [ ]:
from pycaret import time_series

In [ ]:
pivot_by_dates
data_to_exp=pivot_by_dates[2]

In [ ]:
data_to_exp.head()

In [ ]:
fh=7

In [ ]:
#@title análisis exhaustivo de un id específico
id_specifico = 2 #@param {type:"integer"}
df_aux=pivot_by_dates[id_specifico]

time_series.setup( data_to_exp,fh=7 )
####Original dataset
time_series.plot_model()
print("\n"*1)
print(f'id {ts}')
#### ACF and PACF for the original dataset
time_series.plot_model(plot="acf")
print(f'id {ts}')

time_series.plot_model(plot="decomp_classical")
print(f'id {ts}')
try:
    time_series.plot_model(plot="decomp_classical", data_kwargs={'type': 'multiplicative'})
    print(f'id {ts}')
except: 
    pass
time_series.plot_model(plot="decomp_stl")
print(f'id {ts}')
# Show the train-test splits on the dataset
# Internally split - len(fh) as test set, remaining used as test set
time_series.plot_model(plot="train_test_split")
print(f'id {ts}')

# Show the Cross Validation splits inside the train set
time_series.plot_model(plot="cv")
print(f'id {ts}')
# Plot diagnostics
time_series.plot_model(plot="diagnostics")
print(f'id {ts}')
# Plot differences along with diagnostics such as ACF and PACF

# Row 1: Original
# Row 2: d = 1
# Row 3: d = 2
time_series.plot_model(plot="diff", data_kwargs={"order_list": [1, 2], "pacf": True})
print(f'id {ts}')

# Row 1: Original
# Row 2: d = 1
# Row 3: First (d = 1) then (D = 1, s = 12)
#   - Corresponds to applying a standard first difference to handle trend, and
#     followed by a seasonal difference (at lag 12) to attempt to account for
#     seasonal dependence.
# Ref: https://www.sktime.org/en/v0.8.0/api_reference/modules/auto_generated/sktime.transformations.series.difference.Differencer.html
time_series.plot_model(plot="diff", data_kwargs={"lags_list": [[1], [1, 12]], "acf": True, "pacf": True})
print(f'id {ts}')

